In [2]:
from escriptorium_connector import EscriptoriumConnector
from escriptorium_connector.dtos import PostProject, PostDocument, ReadDirection, LineOffset
import os
from dataclasses import dataclass
import json
import io 
from io import BytesIO
import datetime

In [1]:

# eScriptorium instance details
ESCIPTORIUM_URL = 'https://env-9828609.us.reclaim.cloud/'#'http://localhost:8888' 
API_TOKEN = 'c85e1c7e3d081ddd86ee70c58f2d3c66d6e9c6da'#'02835aac5aee131047198e5637b02145ae3e828e'  
USERNAME = 'admin'
PASSWORD = '84a£92oGhC'#'admin'
PROJECT_NAME = 'Gnazim Data'

# Initialize the connector using the API token
connector = EscriptoriumConnector(ESCIPTORIUM_URL, USERNAME, PASSWORD)
#new_project = connector.get_project(connector.get_project_pk_by_name(PROJECT_NAME))
# project_slug = connector.get_project(connector.get_project_pk_by_name(PROJECT_NAME)).slug

NameError: name 'EscriptoriumConnector' is not defined

In [9]:
#getting the xml from escriptorium
bytes_file = connector.download_part_alto_transcription(document_pk=50, part_pk=3143, transcription_pk=94)

In [10]:
import zipfile
import io
import os

def unzip_and_save_xml(zip_bytes, output_folder):
    """
    Unzips the provided bytes (ZIP format) and saves each XML file in the specified folder.
    """
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Use an in-memory file for the zip bytes
    with zipfile.ZipFile(io.BytesIO(zip_bytes)) as zf:
        # Loop over each file in the ZIP
        for file_name in zf.namelist():
            # Check if it's an XML file
            if file_name.endswith('.xml'):
                with zf.open(file_name) as xml_file:
                    # Read XML file contents
                    xml_content = xml_file.read().decode('utf-8')
                    
                    # Define full path for the output XML file
                    output_path = os.path.join(output_folder, file_name)
                    
                    # Write the XML content to a file
                    with open(output_path, 'w', encoding='utf-8') as f:
                        f.write(xml_content)
                    print(f"Saved {file_name} to {output_path}")

# Example usage:

output_folder = "XML Exports"
unzip_and_save_xml(bytes_file, output_folder)


Saved IDGNAZIM0001008.xml to XML Exports\IDGNAZIM0001008.xml
Saved METS.xml to XML Exports\METS.xml


In [14]:
from PIL import Image
import io
import os

def save_image_from_bytes(image_bytes, output_folder, image_name="output_image.png"):
    """
    Converts image bytes to an image and saves it in the specified folder.
    """
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)
    
    # Convert the bytes to an image
    image = Image.open(io.BytesIO(image_bytes))
    
    # Define full path for the output image file
    output_path = os.path.join(output_folder, image_name)
    
    # Save the image (can specify the format here, e.g., PNG, JPEG)
    image.save(output_path)
    print(f"Image saved as {output_path}")

# Example usage:
output_folder = "Image Exports"
save_image_from_bytes(image_bytes, output_folder, "IDGNAZIM0001008.tif")


Image saved as Image Exports\IDGNAZIM0001008.tif


In [12]:
image_bytes = connector.get_document_part_image(doc_pk=50, part_pk=3143,)#.download_part_alto_transcription(document_pk=50, part_pk=3143, transcription_pk=94)

In [9]:
# create new project

# Create a new project
new_project_name = 'persian-model-training'  # Give your project a name
project_slug = new_project_name
project_data = PostProject(name=PROJECT_NAME, slug=new_project_name)  # The slug is usually a URL-friendly version of the name
new_project = connector.create_project(project_data)

print(f"Created new project: {new_project.name}")


Created new project: Persian Model Training


In [4]:
#dont forget to check whther you need to clean xmls from their "word" elements, and if so, use the XML editor notebook

#functions
def upload_images_to_document(connector, folder_path, document_id):
    """Uploads image files to a specified document in eScriptorium."""
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # Add/remove extensions as needed
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'rb') as file:
                # Prepare image data for upload
                image_data = file.read()

                # Create an instance of the dataclass
                image_data_info = ImageDataInfo(filename=filename)

                # Upload the image
                #print(f"Uploading {filename} to document {document_id}...")
                response = connector.create_document_part(
                    document_pk=document_id, 
                    image_data_info=image_data_info, 
                    filename=filename, 
                    image_data=image_data
                )
                print(f"Finished uploading {filename}")

@dataclass
class ImageDataInfo:
    filename: str
    # Add other fields here if needed                
                
def upload_xmls_to_document(connector, folder_path, document_id, transcription_name):
    """Uploads image files to a specified document in eScriptorium."""
    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.xml'):  # Add/remove extensions as needed
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'rb') as file:
                # Prepare image data for upload
                file_data = file.read()

                # Upload the image
                print(f"Uploading {filename} to document {document_id}...")
                response = connector.upload_part_transcription(
                    document_pk=document_id, 
                    transcription_name = transcription_name, 
                    filename=filename, 
                    file_data=file_data,
                    override='off' #change to on if you want to override existing transciptions
                )
                print(f"Finished uploading {filename}")

def iterate_documents(root_folder, connector):
    """
    Iterates through each subfolder in the root_folder. Each subfolder is considered a document.

    :param root_folder: The path to the root folder containing document subfolders.
    """
    # Iterate through each item in the root folder
    for item in os.listdir(root_folder):
        # Construct the full path of the item
        item_path = os.path.join(root_folder, item)
        
        # Check if the item is a directory (i.e., a document folder)
        if os.path.isdir(item_path):
            print(f"Document Name: {item}")
            main_script = "Hebrew"
            document_data = PostDocument(
                name=item,
                project=project_slug,
                main_script=main_script,
                read_direction=ReadDirection.RTL,
                line_offset=LineOffset.BASELINE,
                tags=[]
            )
            #create document
            document_pk = create_document_debug(connector, document_data)
            #upload images
            image_folder = item_path  # Assuming images are directly in the document folder
            upload_images_to_document(connector, image_folder, document_pk)
            #upload page xmls
            transcription_name = f'transcript_{datetime.date.today()}' 
            page_folder = os.path.join(item_path, "page")  # The folder containing XML files
            upload_xmls_to_document(connector, page_folder, document_pk, transcription_name)
            
# Modify the create_document method in EscriptoriumConnector class
def create_document_debug(connector, doc_data):
    url = f"{connector.api_url}documents/"
    # Convert the dataclass to a dictionary
    payload = doc_data.__dict__
    print(f"Request URL: {url}")
    print(f"Payload: {payload}")

    # Perform the POST request
    response = connector.http.post(url, json=payload)

    # Print raw response text
    print(f"Raw Response: {response.text}")
    txt = response.text
    doc_pk = txt[txt.find('":')+2:txt.find(',')]
    return doc_pk 


In [ ]:
root_folder = "C:/Users/User/PycharmProjects/PersianChildhood/Docs to Upload/FarsiPress" # Update this to your actual folder path

#first initialize an instance from an existing project
# connector = EscriptoriumConnector(ESCIPTORIUM_URL, USERNAME, PASSWORD)
# new_project = connector.get_project(connector.get_project_pk_by_name(PROJECT_NAME))
# project_slug = connector.get_project(connector.get_project_pk_by_name(PROJECT_NAME)).slug

iterate_documents(root_folder, connector)


In [8]:
# initialize an instance from an existing project
new_project = connector.get_project(connector.get_project_pk_by_name(PROJECT_NAME))
new_project.slug

'gnazim-trial'

In [10]:
#create a new document

new_document_name = "Gnazim Cards Trial I"
main_script = "Hebrew"


document_data = PostDocument(
    name=new_document_name,
    project=new_project.slug,
    main_script=main_script,
    read_direction=ReadDirection.RTL,
    line_offset=LineOffset.BASELINE,
    tags=[]
)

# Modify the create_document method in EscriptoriumConnector class
def create_document_debug(connector, doc_data):
    url = f"{connector.api_url}documents/"
    # Convert the dataclass to a dictionary
    payload = doc_data.__dict__
    print(f"Request URL: {url}")
    print(f"Payload: {payload}")

    # Perform the POST request
    response = connector.http.post(url, json=payload)

    # Print raw response text
    print(f"Raw Response: {response.text}")
    txt = response.text
    doc_pk = txt[txt.find('":')+2:txt.find(',')]
    return doc_pk 

# Call the modified create_document method
document_pk = create_document_debug(connector, document_data)

Request URL: http://localhost:8080/api/documents/
Payload: {'name': 'Gnazim Cards Trial I', 'project': 'gnazim-trial', 'main_script': 'Hebrew', 'read_direction': <ReadDirection.RTL: 'rtl'>, 'line_offset': <LineOffset.BASELINE: 0>, 'tags': [], '__pydantic_initialised__': True}
Raw Response: {"pk":49,"name":"Gnazim Cards Trial I","project":"gnazim-trial","transcriptions":[{"pk":90,"name":"manual","archived":false,"avg_confidence":null}],"main_script":"Hebrew","read_direction":"rtl","line_offset":0,"show_confidence_viz":false,"valid_block_types":[{"pk":1,"name":"Title"},{"pk":2,"name":"Main"},{"pk":3,"name":"Commentary"},{"pk":4,"name":"Illustration"}],"valid_line_types":[],"valid_part_types":[],"tags":[],"created_at":"2024-04-01T18:25:01.147403Z","updated_at":"2024-04-01T18:25:01.147419Z"}


In [12]:
document_pk

'49'

In [12]:
# Upload images to an existing document
#C:\Users\User\PycharmProjects\PersianChildhood\MJLS_1-20\MJLS_1-20
IMAGE_FOLDER = "training_data/updated_uri_gt_alto"

# ID of the document to which the images will be uploaded
DOCUMENT_ID = 558  # Replace with your document ID

@dataclass
class ImageDataInfo:
    filename: str
    # Add other fields here if needed

def upload_images_to_document(connector, folder_path, document_id):
    """Uploads image files to a specified document in eScriptorium."""
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.tif', '.png', '.jpg', '.jpeg')):  # Add/remove extensions as needed
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'rb') as file:
                # Prepare image data for upload
                image_data = file.read()

                # Create an instance of the dataclass
                image_data_info = ImageDataInfo(filename=filename)

                # Upload the image
                #print(f"Uploading {filename} to document {document_id}...")
                response = connector.create_document_part(
                    document_pk=document_id, 
                    image_data_info=image_data_info, 
                    filename=filename, 
                    image_data=image_data
                )
                print(f"Finished uploading {filename}")

                
# Start the upload process
upload_images_to_document(connector, IMAGE_FOLDER, DOCUMENT_ID)

Finished uploading IDGNAZIM0001.tif
Finished uploading IDGNAZIM00010.tif
Finished uploading IDGNAZIM000100.tif
Finished uploading IDGNAZIM000101.tif
Finished uploading IDGNAZIM000102.tif
Finished uploading IDGNAZIM000103.tif
Finished uploading IDGNAZIM000104.tif
Finished uploading IDGNAZIM000105.tif
Finished uploading IDGNAZIM000106.tif
Finished uploading IDGNAZIM000107.tif
Finished uploading IDGNAZIM000108.tif
Finished uploading IDGNAZIM000109.tif
Finished uploading IDGNAZIM00011.tif
Finished uploading IDGNAZIM000110.tif
Finished uploading IDGNAZIM000111.tif
Finished uploading IDGNAZIM000112.tif
Finished uploading IDGNAZIM000113.tif
Finished uploading IDGNAZIM000114.tif
Finished uploading IDGNAZIM000115.tif
Finished uploading IDGNAZIM000116.tif
Finished uploading IDGNAZIM000117.tif
Finished uploading IDGNAZIM000118.tif
Finished uploading IDGNAZIM000119.tif
Finished uploading IDGNAZIM00012.tif
Finished uploading IDGNAZIM000120.tif
Finished uploading IDGNAZIM000121.tif
Finished uploadin

Finished uploading IDGNAZIM00070.tif
Finished uploading IDGNAZIM00071.tif
Finished uploading IDGNAZIM00072.tif
Finished uploading IDGNAZIM00073.tif
Finished uploading IDGNAZIM00074.tif
Finished uploading IDGNAZIM00075.tif
Finished uploading IDGNAZIM00076.tif
Finished uploading IDGNAZIM00077.tif
Finished uploading IDGNAZIM00078.tif
Finished uploading IDGNAZIM00079.tif
Finished uploading IDGNAZIM0008.tif
Finished uploading IDGNAZIM00080.tif
Finished uploading IDGNAZIM00081.tif
Finished uploading IDGNAZIM00082.tif
Finished uploading IDGNAZIM00083.tif
Finished uploading IDGNAZIM00084.tif
Finished uploading IDGNAZIM00085.tif
Finished uploading IDGNAZIM00086.tif
Finished uploading IDGNAZIM00087.tif
Finished uploading IDGNAZIM00088.tif
Finished uploading IDGNAZIM00089.tif
Finished uploading IDGNAZIM0009.tif
Finished uploading IDGNAZIM00090.tif
Finished uploading IDGNAZIM00091.tif
Finished uploading IDGNAZIM00092.tif
Finished uploading IDGNAZIM00093.tif
Finished uploading IDGNAZIM00094.tif
Fin

In [13]:
# upload page XMLs for existing document and images
#C:\Users\User\PycharmProjects\Arab Tribes\Gazeetter_sample\Gazeetter_sample_new1\Gazeetter_sample_new1
PAGE_FOLDER = "training_data/fixed_uri_gt_alto"

# ID of the document to which the images will be uploaded
DOCUMENT_ID = 558  # Replace with your document ID

def upload_xmls_to_document(connector, folder_path, document_id, transcription_name):
    """Uploads image files to a specified document in eScriptorium."""
    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.xml'):  # Add/remove extensions as needed
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'rb') as file:
                # Prepare image data for upload
                file_data = file.read()

                # Upload the image
                print(f"Uploading {filename} to document {document_id}...")
                response = connector.upload_part_transcription(
                    document_pk=document_id, 
                    transcription_name = transcription_name, 
                    filename=filename, 
                    file_data=file_data,
                    override='off' #change to on if you want to override existing transciptions
                )
                print(f"Finished uploading {filename}")

                

transcription_name = f'Preprocessed XML Import {datetime.date.today()}'
# Start the upload process
upload_xmls_to_document(connector, PAGE_FOLDER, DOCUMENT_ID, transcription_name)

Uploading IDGNAZIM0001.xml to document 558...


RetryError: HTTPSConnectionPool(host='env-9828609.us.reclaim.cloud', port=443): Max retries exceeded with url: /api/documents/558/imports/ (Caused by ResponseError('too many 500 error responses'))

In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())


2.2.1+cu121
True


In [ ]:

def iterate_documents(root_folder, connector):
    """
    Iterates through each subfolder in the root_folder. Each subfolder is considered a document.

    :param root_folder: The path to the root folder containing document subfolders.
    """
    # Iterate through each item in the root folder
    for item in os.listdir(root_folder):
        # Construct the full path of the item
        item_path = os.path.join(root_folder, item)
        
        # Check if the item is a directory (i.e., a document folder)
        if os.path.isdir(item_path):
            print(f"Document Name: {item}")
            main_script = "Arabic"
            document_data = PostDocument(
                name=item,
                project=project_slug,
                main_script=main_script,
                read_direction=ReadDirection.RTL,
                line_offset=LineOffset.BASELINE,
                tags=[]
            )
            # Call the modified create_document method
            document_pk = create_document_debug(connector, document_data)
            
            image_folder = item_path  # Assuming images are directly in the document folder
            
            upload_images_to_document(connector, image_folder, document_pk)
            
            transcription_name = 'transcript'
            
            page_folder = os.path.join(item_path, "page")  # The folder containing XML files
            
            upload_xmls_to_document(connector, page_folder, document_pk, transcription_name)            

In [ ]:
#playing with xml trees

In [ ]:
import xml.etree.ElementTree as ET

def preprocess_page_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    for region in root.findall('.//{*}TextRegion'):
        print(f"TextRegion ID: {region.get('id')}")
        for line in region.findall('.//{*}TextLine'):
            text = line.find('.//{*}Unicode').text
            print(f"  Line Text: {text}")

    for table in root.findall('.//{*}TableRegion'):
        print(f"TableRegion ID: {table.get('id')}")
        for cell in table.findall('.//{*}TableCell'):
            print(f"  TableCell ID: {cell.get('id')}")
            for line in cell.findall('.//{*}TextLine'):
                text = line.find('.//{*}Unicode').text
                print(f"    Line Text: {text}")

preprocess_page_xml(f'{PAGE_FOLDER}/Gazeetter sample-0011.xml')


In [ ]:
import xml.etree.ElementTree as ET
import os
import requests

import xml.etree.ElementTree as ET

def preprocess_page_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Ensure the xmlns attribute is set correctly
    # This example uses a common PAGE XML namespace, adjust as needed
    root.set("xmlns", "http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15")

    # Return the string representation of the modified XML
    return ET.tostring(root, encoding='utf-8').decode('utf-8')

def upload_xml_to_escriptorium(connector, xml_data, document_id, transcription_name):
    # Prepare the payload for upload
    files = {'upload_file': ('transcription.xml', xml_data)}
    payload = {
        'task': 'import-xml',
        'name': transcription_name,
        'override': 'on'  # or 'on' depending on your requirement
    }

    # Perform the upload
    response = connector.upload_part_transcription(
        document_pk=document_id, 
        transcription_name=transcription_name, 
        filename='transcription.xml', 
        file_data=xml_data
    )
    
    return response

def upload_preprocessed_xml_to_document(connector, file_path, document_id, transcription_name):
    # Preprocess the XML
    preprocessed_xml = preprocess_page_xml(file_path)

    # Upload the preprocessed XML to eScriptorium
    response = upload_xml_to_escriptorium(connector, preprocessed_xml, document_id, transcription_name)
    
    if response.status_code == 200:
        print("Upload successful.")
    else:
        print(f"Upload failed with status code {response.status_code}: {response.text}")

# Example usage
file_path = f'{PAGE_FOLDER}/Gazeetter sample-0011.xml'  # Replace with your file path
document_id = '3'  # Replace with your document ID
transcription_name = 'your_transcription_name'  # Replace with your transcription name

upload_preprocessed_xml_to_document(connector, file_path, document_id, transcription_name)


In [ ]:
# for filename in os.listdir(PAGE_FOLDER)[-10:]:
#     if filename.lower().endswith('.xml'):  # Add/remove extensions as needed
#         file_path = os.path.join(PAGE_FOLDER, filename)
#         with open(file_path, 'rb') as file:
#             # Prepare image data for upload
#             file_data = file.read()

#         # Read the Page XML file
# with open(file_path, 'rb') as file:
#     file_data = file.read()

# Upload the Page XML
response = connector.upload_part_transcription(
    document_pk=DOCUMENT_ID, 
    transcription_name="MyTranscription", 
    filename=filename, 
    file_data=BytesIO(file_data),
    override='on'  # Use 'on' if you want to override existing data
)